In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### 1

В чём идея параметрических статистических критериев, которые мы рассматривали на прошлом уроке?

Мы предполагаем вид распределения случайной величины, чтобы выбрать подходящую статистику, распределение которой мы знаем

### 2

В чём идея непараметрических статистических критериев?

От случайной величины с неизвестным параметром мы переходим к новой, распределение которой мы знаем

### 3

Какие гипотезы проверяет критерий знаков?
- Равенство медианы выборки некоторому числу
- Равенство дисперсии выборки некоторому числу
- Равенство среднего выборки некоторому числу

Критерий знаков делает переход к новой величине — индикатору того, что значение больше этого некоторого значения. Соответственно нулевая гипотеза в том, является ли это некоторое значение медианой выборки.

### 4

Однажды мы с вами сравнивали метрики между двумя моделями на одной и той же выборке. 

Чаще всего метрики по объектам представляют собой ненормальное распределение, поэтому лучше оценивать их различия непараметрическими статистическими критериями. Давайте попрактикумемся в этом. Возьмём уже привычный нам датасет со средними ценами на дома.

Обучите Ridge-регрессии из библиотеки sklearn с параметрами alpha=1 и alpha=10. Предсказания сделайте на всей выборке по фолдам с помощью функции cross_val_predict (по очереди каждый фолд будет становиться тестовым), не забудьте передать туда наше разбиение.

Посчитайте MSE по каждому объекту выборки и сравните равенство их медиан с помощью критерия знаков. Сам критерий знаков реализуйте вручную, его нулевое распределение представляет собой биномиальное распределение, с котором мы уже работали ранее.

Какой вывод вы можете сделать о различии этих моделей? Альтернатива двусторонняя, уровень значимости 0.05.

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

data = fetch_california_housing(as_frame=True, return_X_y=True)
df, target = data[0], data[1] 
df = StandardScaler().fit_transform(df)

cv = KFold(n_splits=4, shuffle=True, random_state=6)

Сделаем предсказания моделями, обученными по k-фолдовой кросс-валидации:

In [3]:
from sklearn.linear_model import Ridge, Lasso

y_ridge_1 = cross_val_predict(Ridge(alpha=1.0), X=df, y=target, cv=cv)
y_ridge_2 = cross_val_predict(Ridge(alpha=10.0), X=df, y=target, cv=cv)

Оценим качество:

In [4]:
mse_ridge_1 = (y_ridge_1 - target) ** 2
mse_ridge_2 = (y_ridge_2 - target) ** 2

np.mean(mse_ridge_1), np.mean(mse_ridge_2)

(0.5305334005667641, 0.5304527930834362)

Перейдём к новой случайной величине для критерия знаков:

In [5]:
values = (mse_ridge_1 > mse_ridge_2).astype(int)

np.mean(values)

0.4810077519379845

Оценим значимость (здесь мы учитываем, что среднее значение в выборке меньше 0.5, то есть наша статистика слева от центра):

In [6]:
from scipy.stats import binom

binom(n=len(values), p=0.5).cdf(sum(values)) * 2

5.0154276353317846e-08

Ну или автоматически:

In [7]:
from scipy.stats import binom_test

binom_test(x=sum(values), n=len(values))

/Users/emilkayumov/venv/lib/python3.8/site-packages/scipy/stats/_discrete_distns.py:69: RuntimeWarning: divide by zero encountered in _binom_pdf
  return _boost._binom_pdf(x, n, p)


5.0154276353317846e-08

То есть p-value=0, метрики между моделями значимо различаются

### 5

Некоторые непараметрические статистические критерии делают переход к рангам.

Посчитайте ранги для элементов следующей выборки. Ранги равных по значениям элементов считайте так же, как мы это делали на уроке.

[7, 1, 5, 1, 3, 2, 5]

**Решение**

[7, 1.5, 5.5, 1.5, 4, 3, 5.5]

Также можно вычислять автоматически (pandas тоже умеет). Важно выбрать в аргументе то, что делать с равными значениями.

In [8]:
import scipy.stats as ss

ss.rankdata([7, 1, 5, 1, 3, 2, 5], method='average')

array([7. , 1.5, 5.5, 1.5, 4. , 3. , 5.5])

### 6

Какую гипотезу проверяет критерий Манна-Уитни-Уилкоксона?

Равенство функций распределения в двух выборках

### 7

В критерии Манна-Уитни-Уилкоксона распределение статистики табличное. Статистикой выступает сумма рангов первой выборки в объединённой выборке.

Представим, что у нас две выборки: первая размера 2, а вторая размера 3. Все значения в выборках различаются.

Постройте соответствующее этому случаю табличное распределение и ответьте на вопрос, с какой вероятностью мы могли встретить значения статистики 7 и более среди всех возможных значений статистики при верности нулевой гипотезы о том, что распределения не различаются.

Это задание удобнее делать на листочке без написания кода.

**Решение**

Запишем все возможные варианты расположения первой выборки в объединённой выборке из 5 элементов:

- [1, 2] — 3
- [1, 3] — 4
- [1, 4] — 5
- [1, 5] — 6
- [2, 3] — 5
- [2, 4] — 6
- [2, 5] — 7
- [3, 4] — 7
- [3, 5] — 8
- [4, 5] — 9

То есть значение 7 и более встречается 4 раза из 10 возможных, то есть 0.4.

### 8

В чём неудобство работы с табличными распределениями?

Табличное распределение необходимо посчитать для всех возможных значений и хранить большое количество значений. При этом посчитать точно всё-таки возможно и использовать его можно (не только теоретическая конструкция).

### 9

Вернёмся к уже привычному для нас датасету со средними ценами на дома, с которым мы работали в прошлых заданиях. Для скачивания воспользуйтесь тем же кодом:

В прошлом уроке мы сравнивали средние доходы (MedInc) между районами, где дома моложе 30 лет (включительно) и старше 30 лет (HouseAge). Однако тогда мы заметили, что случайная величина распределена не нормально.

Давайте воспользуемся непараметрическими статистическими критериями, которые не делают предположений на вид распределения.

Примените критерий Манна-Уитни-Уилкоксона к выборке и сравните, различаются ли распределения между выборками. Какой вывод вы можете сделать? Альтернатива двусторонняя.

In [9]:
from sklearn.datasets import fetch_california_housing

data = fetch_california_housing(as_frame=True, return_X_y=True)
df, target = data[0], data[1] 

Снова разделим выборку на 2:

In [10]:
df1 = df[df.HouseAge <= 30]
df2 = df[df.HouseAge > 30]

Применим критерий:

In [11]:
from scipy.stats import mannwhitneyu

mannwhitneyu(df1.MedInc, df2.MedInc)

MannwhitneyuResult(statistic=59254034.5, pvalue=5.345198689020035e-50)

Мы получили p-value, который близко к нулю, а значит можем отвергнуть нулевую гипотезу на уровне значимости 0.05, что распределения случайных величин между выборками не различаются. Таким образом можно сказать, что средние доходы разные.

### 10

Какую проблему решают подходы к множественной проверке гипотез?

Когда мы проверяем сразу несколько гипотез, то уровень ошибки первого рода возрастает

### 11

Представим, что мы хотели бы зафиксировать ошибку первого рода при применении статистического критерия на уровне 1%. У нас есть 12 экспериментальных моделей, среди которых мы хотим найти те, которые будут отличаться от текущей в лучшую сторону.

Какая ошибка первого рода «хотя бы одна из моделей случайно покажется лучше контрольной» у нас будет на самом деле? Результаты всех моделей независимы друг от друга.

**Решение**

Посчитаем эту вероятность.

Сначала посчитаем вероятность не ошибиться ни разу в каждом из 12 экспериментов:

In [12]:
(1 - 0.01) ** 12

0.8863848717161292

Тогда ошибиться хотя бы раз по свойству вероятности протовоположного события:

In [13]:
1 - (1 - 0.01) ** 12

0.11361512828387077

### 12

Выберите верные утверждения про FWER-поправки при множественное проверке гипотез.
1. При применении поправок уменьшается пороговое значение p-value
1. При применении поправок увеличивается пороговое значение p-value
1. При их применении возрастает ошибка второго рода (ложноотрицательные ошибки)
1. Поправки контролируют уровень ошибки второго рода (ложноотрицательные ошибки)
1. При их применении возрастает ошибка первого рода (ложноположительные ошибки)
1. Поправки контролируют уровень ошибки первого рода (ложноположительные ошибки)

**Решение**

FWER — процедура для контроля за ошибками первого рода, которая изменяет пороговые значения p-value в меньшую сторону, тем самым возрастает ошибка второго рода.

Пороговое значение p-value уменьшается в каждом конкретном эксперименте (1 верно, 2 не верно).

Из-за понижения порогового значения p-value ошибка второго рода растёт (3 верно, 4 не верно)

Сделаны ни как раз для контроля за ошибкой первого рода (5 не верно, 6 верно).